In [1]:
!pip install retrying
!pip install fake_useragent
from bs4 import BeautifulSoup
from retrying import retry
import requests
import pandas as pd
import sys
import time
import random
from urllib.parse import quote
from fake_useragent import UserAgent



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
URL = "https://www.amazon.com/Apple-iPhone-11-64GB-Black/dp/B07ZPKN6YR/ref=sr_1_1?crid=39DAZO9YLWW6V&keywords=iphone&qid=1700427702&sprefix=iphone%2Caps%2C331&sr=8-1&th=1"
HEADERS  = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

# Créez une instance de UserAgent
ua = UserAgent()

In [4]:
webpage = requests.get(URL, headers=HEADERS)
print(webpage.status_code)

200


In [6]:
brew install python@3.10


SyntaxError: invalid syntax (716452758.py, line 1)

In [5]:
!python --version


Python 3.11.5


In [6]:
soup = BeautifulSoup(webpage.content, "html.parser")
reviews = []
link = []
for b in soup.findAll("a", {'data-hook': "see-all-reviews-link-foot"}):
  link.append(b['href'])
print("links", link)

links ['/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', '/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews']


In [7]:
# Function to make a request using Crawlbase
CRAWLBASE_TOKEN = '6CtC079r3M21jspX_DPN2g'  # Your Crawlbase token
def make_request(url):
    encoded_url = quote(url, safe='')
    crawlbase_url = f'https://api.crawlbase.com/?token={CRAWLBASE_TOKEN}&url={encoded_url}'
    response = requests.get(crawlbase_url, headers={'User-Agent': ua.random})
    return response

In [8]:
from concurrent.futures import ThreadPoolExecutor

# Your other functions and imports...

MAX_RETRIES = 2
RETRY_SLEEP_SECONDS = 2

def make_request_with_retry(url):
    for attempt in range(MAX_RETRIES):
        page = make_request(url)

        if page.status_code == 200:
            return page
        else:
            print(f"Error fetching page {url} - Attempt {attempt + 1} - Status Code: {page.status_code}")
            #print("Response Body:", page.text)
            time.sleep(RETRY_SLEEP_SECONDS)

    print(f"Failed to fetch page {url} after {MAX_RETRIES} attempts.")
    return None

def process_page(url):
    page = make_request_with_retry(url)

    if page is not None and page.status_code == 200:
        print("Page", url, "fetched successfully")
        soup = BeautifulSoup(page.content, "html.parser")

        new_reviews = [i.text for i in soup.findAll("span", {'data-hook': "review-body"})]
        if not new_reviews:
            print("No new reviews found on page", url)
            return None

        time.sleep(random.uniform(2, 4))
        return new_reviews
    else:
        return None

if link:
    new_link = "https://www.amazon.com" + link[0]
    urls = [new_link + f'&pageNumber={k}' for k in range(1, 13)]  # Adjust the range as needed

    with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust the number of workers as needed
        reviews = list(filter(None, executor.map(process_page, urls)))

    if reviews:
        reviews = [review for sublist in reviews for review in sublist]
        rev = {'reviews': reviews}
        review_data = pd.DataFrame.from_dict(rev)
        review_data.to_csv("amazon_data.csv", header=True, index=False)
        print("Total reviews collected:", len(reviews))
    else:
        print("No reviews collected.")
else:
    print("Error fetching initial page - Status Code:", webpage.status_code)

ConnectionError: HTTPSConnectionPool(host='api.crawlbase.com', port=443): Max retries exceeded with url: /?token=6CtC079r3M21jspX_DPN2g&url=https%3A%2F%2Fwww.amazon.com%2FApple-iPhone-11-64GB-Black%2Fproduct-reviews%2FB07ZPKN6YR%2Fref%3Dcm_cr_dp_d_show_all_btm%3Fie%3DUTF8%26reviewerType%3Dall_reviews%26pageNumber%3D1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x15c7dcb10>: Failed to resolve 'api.crawlbase.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [2]:
pip install urllib3==2.0.7


  Obtaining dependency information for urllib3==2.0.7 from https://files.pythonhosted.org/packages/d2/b2/b157855192a68541a91ba7b2bbcb91f1b4faa51f8bae38d8005c034be524/urllib3-2.0.7-py3-none-any.whl.metadata
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip list 

Package                       Version
----------------------------- ------------
aiobotocore                   2.5.0
aiofiles                      22.1.0
aiohttp                       3.8.5
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.3
anaconda-navigator            2.5.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
applaunchservices             0.3.0
appnope                       0.1.2
appscript                     1.1.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
async-timeout                 4.0.2
atomicwrites 

In [8]:
print(review_data)

NameError: name 'review_data' is not defined

In [23]:
# Afficher un review en entier
if not review_data.empty:
    review = review_data['reviews'].iloc[2]
    print("Full text of the first review:\n", review)
else:
    print("Aucune revue n'a été collectée.")

Full text of the first review:
 
I'll be honest with you, at first I was a bit skeptical. I was told to avoid "Renewed" phones because unlike Apple-refurbished phones, different refurbishment factories are sourced online, and the fact there are only stock images rather than actual pictures of the phone someone took. My first concern was from the other reviews I've read that range from 70% battery capacity, to detecting random touch screen inputs, to speakers not working, to vibration modules not working at all, which seemed like buying refurb phones from the Internet is a pretty big gamble. Fortunately, and thankfully, I have found a diamond in the rough. This was a refurb I bought from Stone Digital. This 11 didn't bug me about "non-genuine Apple parts" messages, didn't have touch screen problems (at least I put the screen protector on before turning it on for the first time if that helps), Speakers (both external and head speaker) worked fine, SIM card reader worked fine, Battery Cap